# Fixed-points of Boolean networks in AEON.py

AEON.py includes methods for reasonably efficient symbolic computation of network fixed-points. 
Note that if we are only interested in *some* fixed-points, for very large models, it's probably 
faster to use some kind of SAT/ASP based method. However, if we need to further explore the full 
set of fixed-points, e.g. to compute it's cardinality, it might be infeasible to enumerate them all 
using just a solver. That's where AEON.py comes in.

To demonstrate the feature, we will use the butanol production pathway model from CellCollective (also known as model 077 in [BBM](https://github.com/sybila/biodivine-boolean-models/tree/94e521651376fa7d6979a4aebc1df574de0d5c2c/models/%5Bid-077%5D__%5Bvar-53%5D__%5Bin-13%5D__%5BSIGNALLING-PATHWAY-FOR-BUTANOL-PRODUCTION%5D)).

In [1]:
from biodivine_aeon import *

In [2]:
# The `from_file` function will automatically recognise 
# the network format from the file extension.
bn = BooleanNetwork.from_file("butanol-pathway.sbml")
print(bn)

BooleanNetwork(variables = 66, parameters = 0, regulations = 152)


#### Input inlining (optional)

At this point, all model inputs are represented as variables with constant update functions
(this representation tends to have the best compatibility with other tools). However, this is a bit 
inefficient for BDD representation, because while the inputs are clearly constant, AEON will still treat them as normal variables.

Instead, we can ask AEON to automatically recognize constant input 
variables and turn them into logical parameters. Overall, this is probably not going to turn an unsolvable
problem into a solvable one, but it can still save a non-trivial amount of computation time (and quite a bit of RAM
for larger models).
 
However, keep in mind that AEON refers to the valuations of such parameters as colors: if the model is using any parameters, the result won't be just a set of fixed-point states, but a relation over fixed-point states and corresponding parameter valuations (colors). 

If you don't perform this transformation, you don't have to worry about colors at all. Your models won't contain any and all information will be part of the network state (as long as you don't introduce other parameters, e.g. by erasing the update functions of other variables).

In [3]:
bn_inlined = bn.inline_inputs()
print(bn_inlined)

BooleanNetwork(variables = 53, parameters = 13, regulations = 120)


We will continue working with both versions of the network to demonstrate the differences between a "normal" Boolean network and one with logical parameters. However, keep in mind that the results will be isomorphic: the only difference is whether parts of the result identify as a "state" or a "color".

#### Symbolic transition graph

For the next step, we need to create the `SymbolicAsyncGraph` for both networks. This structure actually encodes the network behaviour into a symbolic transition system:

In [4]:
stg = SymbolicAsyncGraph(bn)
print(f"STG has {int(stg.unit_vertices().cardinality())} states and {int(stg.unit_colors().cardinality())} color.")
stg_inlined = SymbolicAsyncGraph(bn_inlined)
print(f"STG has {int(stg_inlined.unit_vertices().cardinality())} states and {int(stg_inlined.unit_colors().cardinality())} colors.")

STG has 73786976294838206464 states and 1 color.
STG has 9007199254740992 states and 8192 colors.


Notice that the number of states in the "normal" network matches the number of states times the number of colors of the inlined network. That is, we are not losing any information, some of the state variables just shifted into logical parameters.

### Solver-based fixed-point enumeration

First, we show how to use a solver in AEON.py to detect fixed-points. Internally, this relies on Z3, 
which is a bit of an overkill for this kind of task but is relatively reliable and easy to integrate. 

 > The solver is statically linked into AEON.py, so you don't have to install Z3 independently.

 > WARNING: Right now, the result is exported to Python as a list, not an iterator. Internally 
   (in Rust), the result is an iterator, but porting Rust iterators into Python is a combersome process 
   that we plan to adress later. Just keep in mind that all results have to fit into memory in this case. 
   If this is a limitation for you (i.e. you want a true iterator), get it touch and we can fix this :)

In [5]:
fixed_point_list = FixedPoints.solver_list(stg, limit=100000)
print(len(fixed_point_list))
fixed_point_list_inlined = FixedPoints.solver_list(stg_inlined, limit=10000)
print(len(fixed_point_list_inlined))

2048
2048


Note that the results of this enumeration (members of the list) are still symbolic sets. They just all 
contian a single state and a single color. This is mostly to ensure compatibility with other parts 
of the AEON.py API. This also means you can treat them as sets and perform operations like 
`union` or `intersection` on them:

In [6]:
# Make sure not to mix the symbolic sets from `stg` and `stg_inlined`.
# While they technically have the same number of symbolic variables,
# they are not guaranteed to be mutually compatible.

solver_result = stg_inlined.empty_colored_vertices()
for x in fixed_point_list_inlined:
    # `x` is a singleton set: it contains just one element.
    assert x.is_singleton()
    
    solver_result = solver_result.union(x)
print(int(solver_result.cardinality()))

2048


### Symbolic fixed-point calculation

If a network has *a lot* of fixed-points, it might be impossible to enumerate them all using a solver. 
That's where a symbolic algorithm can be useful, but keep in mind that the computation can still take a long time for large networks.

In [7]:
fp = FixedPoints.symbolic(stg)
print(int(fp.cardinality()))
fp_inlined = FixedPoints.symbolic(stg_inlined)
print(int(fp_inlined.cardinality()))

# Naturally, the results should be the same as the ones from the solver.
assert fp_inlined == solver_result

2048
2048


This is also where the colors come into play. We can notice that the result for both networks contains 2048 elements, but these are distributed differently between vertices and colors. In particular, we see that there are only 32 unique states which appear for 2048 unique colors (parameter valuations). This means that 2048/8192 colors admit a fixed point, but out of these 2048 colors, some colors admit the same fixed-point, since there are only 32 unique fixed-point states.

In [8]:
print(f"{int(fp.vertices().cardinality())} vertices, {int(fp.colors().cardinality())} colors")
print(f"{int(fp_inlined.vertices().cardinality())} vertices, {int(fp_inlined.colors().cardinality())} colors")

2048 vertices, 1 colors
32 vertices, 2048 colors


In [9]:
# We can even iterate over the vertices and print them all out:
for vertex in fp_inlined.vertices().iterator():
    print(vertex[:10], "...")

[False, True, False, True, False, False, False, False, False, False] ...
[False, True, True, True, False, False, False, False, False, False] ...
[False, True, False, True, False, False, False, False, False, False] ...
[False, True, True, True, False, False, False, False, False, False] ...
[False, True, False, True, False, False, False, False, False, False] ...
[False, True, True, True, False, False, False, False, False, False] ...
[False, True, False, True, False, False, False, False, False, False] ...
[False, True, True, True, False, False, False, False, False, False] ...
[True, True, False, True, False, False, True, True, False, True] ...
[True, True, True, True, False, False, True, True, False, True] ...
[True, True, False, True, False, False, True, True, False, True] ...
[True, True, True, True, False, False, True, True, False, True] ...
[True, True, False, True, False, False, True, True, False, True] ...
[True, True, True, True, False, False, True, True, False, True] ...
[True, Tr

### Advanced fixed point functionality

#### Restriction to initial states

We can use a `restriction` parameter (this can be any symbolic `ColoredVertexSet`) that restricts the search only to this particular subset of the state space. For example, we might be interested in the distribution of the fixed-points depending on the `sporulation` variable:

In [10]:
symbolic_space = stg.fix_subspace({ "v_sporulation": False })
fixed_points = FixedPoints.symbolic(stg, restriction=symbolic_space)
print(f"Found {int(fixed_points.cardinality())} fixed-points")

symbolic_space = stg.fix_subspace({ "v_sporulation": True })
fixed_points = FixedPoints.symbolic(stg, restriction=symbolic_space)
print(f"Found {int(fixed_points.cardinality())} fixed-points")

Found 0 fixed-points
Found 2048 fixed-points


Ha! Indeed, it appears that all fixed points have `sporulation` set to `True`. 

Note that since we already had the complete symbolic set of fixed-points (variables `fp` and `fp_inlined`), we could have gotten the same result 
just by intersecting the `fp` set with the two versions of the `symbolic_space`. However, for more restrictive sets of "initial" states, it is often faster to just compute the two results independently (assuming the full result is not needed).

#### Projection to individual components

In some instances, we only care about the fixed-point states (without the colors; i.e. parameter valuations), 
or about the fixed-point colors (parameter valuations that admit existence of *some* fixed-point). In these cases,
we can use a projection to only obtain this particular part of the result. Similar to restrictions, this is typically measurably faster than computing the full result.

In [11]:
fp_vertices = FixedPoints.symbolic_vertices(stg_inlined)
fp_colors = FixedPoints.symbolic_colors(stg_inlined)

assert fp_vertices == fp_inlined.vertices()
assert fp_colors == fp_inlined.colors()

However, keep in mind that the set of fixed points is more complex than just 
the product of the vertex and color projection. That is, we cannot simply do a cartesian product of `fp_vertices` with `fp_colors` and expect to obtain the `fp_inlined` set.

 > Right now, AEON.py can only do projections to vertices (`VertexSet`) and colors (`ColorSet`). However, the underlying Rust code can actually do projections to any subset of variables or parameters. The main limitation is that we don't have a type safe way of manipulating such projections in Python yet. If you want to use such feature for something, get in touch and we can add it :)